<a href="https://colab.research.google.com/github/abx67/Applied-Deep-Learning/blob/master/fan_tumor_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is used to localize tumor area for `tumor_091.tif`.  

Done(12.19)----------> Improvement:
1. `kp_image.img_to_array()` preprocess images more efficiently. 
2. `np.asarray()` would not copy array like `np.array()`.
3. `model`: VGG16+global pooling 2D instead of flatten layer. 
4. `label2mask`: change i with j because width and height issue.

---

TODO(12.19):
1. **find tissue areas improvement**. (do not use too many non-tissue areas so as to improve performance.)
2. Evaluation: check accuracy! 



In [0]:
import tensorflow as tf
tf.enable_eager_execution()

In [2]:
# Install the OpenSlide C library and Python bindings
!apt-get install openslide-tools
!pip install openslide-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openslide-tools is already the newest version (3.4.1+dfsg-2).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from openslide import open_slide, __library_version__ as openslide_version
import os
from PIL import Image
from skimage.color import rgb2gray
from tensorflow.python.keras.preprocessing import image as kp_image

## Step 0: Download Slide and Show Details

In [0]:
slide_path = 'tumor_091.tif'
tumor_mask_path = 'tumor_091_mask.tif'

slide_url = 'https://storage.googleapis.com/applied-dl/%s' % slide_path
mask_url = 'https://storage.googleapis.com/applied-dl/%s' % tumor_mask_path

# Download the whole slide image
if not os.path.exists(slide_path):
  !curl -O $slide_url

# Download the tumor mask
if not os.path.exists(tumor_mask_path):
  !curl -O $mask_url

In [5]:
slide = open_slide(slide_path)
print ("Read WSI from %s with width: %d, height: %d" % (slide_path, 
                                                        slide.level_dimensions[0][0], 
                                                        slide.level_dimensions[0][1]))

tumor_mask = open_slide(tumor_mask_path)
print ("Read tumor mask from %s" % (tumor_mask_path))

print("Slide includes %d levels", len(slide.level_dimensions))
for i in range(len(slide.level_dimensions)):
    print("Level %d, dimensions: %s downsample factor %d" % (i, 
                                                             slide.level_dimensions[i], 
                                                             slide.level_downsamples[i]))
    assert tumor_mask.level_dimensions[i][0] == slide.level_dimensions[i][0]
    assert tumor_mask.level_dimensions[i][1] == slide.level_dimensions[i][1]

# Verify downsampling works as expected
width, height = slide.level_dimensions[7]
assert width * slide.level_downsamples[7] == slide.level_dimensions[0][0]
assert height * slide.level_downsamples[7] == slide.level_dimensions[0][1]

Read WSI from tumor_091.tif with width: 61440, height: 53760
Read tumor mask from tumor_091_mask.tif
Slide includes %d levels 8
Level 0, dimensions: (61440, 53760) downsample factor 1
Level 1, dimensions: (30720, 26880) downsample factor 2
Level 2, dimensions: (15360, 13440) downsample factor 4
Level 3, dimensions: (7680, 6720) downsample factor 8
Level 4, dimensions: (3840, 3360) downsample factor 16
Level 5, dimensions: (1920, 1680) downsample factor 32
Level 6, dimensions: (960, 840) downsample factor 64
Level 7, dimensions: (480, 420) downsample factor 128


## Step 1: Read Slide and Preprocess Data

In [0]:
def read_slide(slide, x, y, level, width, height, as_float=False):
    im = slide.read_region((x,y), level, (width, height))
    im = im.convert('RGB') # drop the alpha channel
    return im

In [0]:
def data_aug(x):
  
  #rotate 4 directions and left-right flip
  x1 = np.rot90(x,k=1)
  x2 = np.rot90(x1,k=1)
  x3 = np.rot90(x2,k=1)
  x4 = np.rot90(x3,k=1)
  x5 = x1[:, ::-1]
  x6 = x2[:,::-1]
  x7 = x3[:,::-1]
  x8 = x4[:,::-1]
  x_new= [x1,x2,x3,x4,x5,x6,x7,x8]
  
  return x_new

In [0]:
# # Version 1.0: save images to disk
# def data_preprocess(slide, tumor_mask, level, window_size=(299,299),
#                            center_size=(128,128),stride=128):
  
#   if not os.path.exists('train'): 
#     os.mkdir('train')
#   if not os.path.exists('train/tumor'): 
#     os.mkdir('train/tumor')
#   if not os.path.exists('train/normal'):
#     os.mkdir('train/normal')
  
#   factor = int(slide.level_downsamples[level])
#   tumor_count = 1
#   normal_count = 1
#   x_step = (window_size[0]-center_size[0])//2
#   y_step = (window_size[1]-center_size[1])//2
#   imgs0 = []
#   imgs1 = []
  
#   for i in range(0, slide.level_dimensions[level][0]-window_size[0], stride):  #width
#     for j in range(0, slide.level_dimensions[level][1]-window_size[1], stride):   #height
#       slide_img = read_slide(slide, i*factor, j*factor, level, window_size[0], window_size[1], as_float=False)
#       mask_img = read_slide(tumor_mask, (i+x_step)*factor, (j+y_step)*factor, level, center_size[0], center_size[1], as_float=False)
#       if np.sum(mask_img)==0:   #normal
#         #slide_img.save("train/normal/%d.jpg" % normal_count)
#         #normal_count += 1
#         if np.random.uniform(size=1)>0.6:
#           imgs0.append(kp_image.img_to_array(slide_img)/255)   #np.asarray?
#           #labels.append(0.)
#       elif np.sum(mask_img)!=0: #tumor
#         slide_img.save("train/tumor/%d.jpg" % tumor_count)
#         tumor_count += 1
#         imgs1 += data_aug(kp_image.img_to_array(slide_img)/255)
        
#   imgs = np.concatenate((np.asarray(imgs0),np.asarray(imgs1)))
#   labels = np.append(np.repeat(0.,len(imgs0)),np.repeat(1.,len(imgs1)))
#   ind = [i for i in range(labels.shape[0])]
#   np.random.shuffle(ind)
  
#   return imgs[ind], labels[ind]

In [0]:
# Version 2.0
def data_preprocess(slide, tumor_mask, level, window_size=(299,299),
                           center_size=(128,128),stride=128):
  
  factor = int(slide.level_downsamples[level])
  tumor_count = 1
  normal_count = 1
  x_step = (window_size[0]-center_size[0])//2
  y_step = (window_size[1]-center_size[1])//2
  imgs0 = []
  imgs1 = []
  
  for i in range(0, slide.level_dimensions[level][0]-window_size[0], stride):  
    for j in range(0, slide.level_dimensions[level][1]-window_size[1], stride): 
      slide_img = read_slide(slide, i*factor, j*factor, level, window_size[0], window_size[1], as_float=False)
      mask_img = read_slide(tumor_mask, (i+x_step)*factor, (j+y_step)*factor, level, center_size[0], center_size[1], as_float=False)
      if np.sum(mask_img)==0:   #normal
        if np.random.uniform(size=1)>0.75: #0.75 has to be tuned each time when we change the size!
          imgs0.append(kp_image.img_to_array(slide_img)/255)  
      elif np.sum(mask_img)!=0: #tumor
        imgs1 += data_aug(kp_image.img_to_array(slide_img)/255)
        
  imgs = np.concatenate((np.asarray(imgs0),np.asarray(imgs1)))
  labels = np.append(np.repeat(0.,len(imgs0)),np.repeat(1.,len(imgs1)))
  ind = [i for i in range(labels.shape[0])]
  np.random.shuffle(ind)
  
  return imgs[ind], labels[ind]

In [0]:
window_size=(80,80)
center_size=(50,50)
stride=50
level=3

In [0]:
patches, labels= data_preprocess(slide, tumor_mask, level=3, window_size=window_size,
                           center_size=center_size,stride=stride)

In [12]:
np.unique(labels,return_counts=True)

(array([0., 1.]), array([5044, 3376]))

## Step 2: Train a Model

In [0]:
model = tf.keras.applications.VGG16(input_shape=(299, 299, 3),include_top=False)
model.trainable = False
x = model.output
x = tf.keras.layers.Dense(128,activation='relu')(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
predictions = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model_baseline = tf.keras.models.Model(inputs = model.input, outputs = predictions)
model_baseline.compile(loss ="binary_crossentropy",
                       optimizer = tf.train.RMSPropOptimizer(learning_rate=0.0001),
                       metrics=["accuracy"])

In [14]:
model_baseline.fit(patches,labels,epochs=10,batch_size=32,validation_split=0.25)

Epoch 1/10
198/198 [==============================] - 48s 242ms/step - loss: 0.3887 - acc: 0.8067 - val_loss: 0.2086 - val_acc: 0.9093
Epoch 2/10
198/198 [==============================] - 44s 222ms/step - loss: 0.2197 - acc: 0.9012 - val_loss: 0.1285 - val_acc: 0.9430
Epoch 3/10
198/198 [==============================] - 44s 221ms/step - loss: 0.1800 - acc: 0.9249 - val_loss: 0.1075 - val_acc: 0.9530
Epoch 4/10
198/198 [==============================] - 44s 221ms/step - loss: 0.1302 - acc: 0.9501 - val_loss: 0.0983 - val_acc: 0.9582
Epoch 5/10
198/198 [==============================] - 44s 220ms/step - loss: 0.1114 - acc: 0.9640 - val_loss: 0.1375 - val_acc: 0.9515
Epoch 6/10
198/198 [==============================] - 44s 220ms/step - loss: 0.0931 - acc: 0.9706 - val_loss: 0.0823 - val_acc: 0.9734
Epoch 7/10
164/198 [=======================>......] - ETA: 6s - loss: 0.0872 - acc: 0.9724

KeyboardInterrupt: ignored

In [0]:
model_baseline.save_weights('baseline_80_50.h5')

In [0]:
# # if running time died, please run this step after building model directly.
# model_baseline.load_weights('baseline_80_50.h5')

### Step 2.1 weighted loss

In [0]:
from functools import partial, update_wrapper

def wrapped_partial(func, *args, **kwargs):
	partial_func = partial(func, *args, **kwargs)
	update_wrapper(partial_func, func)
	return partial_func

def binary_crossentropy_weigted(y_true, y_pred, class_weights):
	y_pred = K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon())
	loss = K.mean(class_weights*(-y_true * K.log(y_pred) - (1.0 - y_true) * K.log(1.0 - y_pred)),axis=-1)
	return loss

custom_loss = wrapped_partial(binary_crossentropy_weigted, class_weights=np.array([1.0, 2.0]))

### Step 2.2 multi-scale

In [0]:
### layers defination

input_shape_high = (300, 300, 3)
input_shape_low = (150, 150, 3)


input4 = tf.keras.layers.Input(shape = input_shape_high)
layer_level4 = tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape_high,
                 padding='same')(input4)
layer_level4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(layer_level4)
layer_level4 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                                    activation='relu',
                                    padding='same')(layer_level4)
layer_level4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(layer_level4)

input5 = tf.keras.layers.Input(shape = input_shape_low)
layer_level5 = tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation = 'relu',
                 input_shape = input_shape_low,
                 padding = 'same')(input5)
layer_level5 = tf.keras.layers.MaxPooling2D(pool_size = (2, 2))(layer_level5)
layer_level5 = tf.keras.layers.Conv2D(64, kernel_size = (3, 3),
                                    activation = 'relu',
                                    padding = 'same')(layer_level5)

added = tf.keras.layers.Add()([layer_level4, layer_level5])

combine_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(added)
combine_layer = tf.keras.layers.Flatten()(combine_layer)
combine_layer = tf.keras.layers.Dense(128, activation ='relu')(combine_layer)
out = tf.keras.layers.Dense(3, activation='softmax')(combine_layer)

combine_model = tf.keras.models.Model(inputs=[input4, input5], outputs=out)

combine_model.compile(optimizer=tf.train.RMSPropOptimizer(learning_rate=0.001),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

## Step 3: Prediction for Trained Slide

In [0]:
## convert labels to mask
def label2mask(y_pred, mask_size, window_size=(150,150),center_size=(100,100),stride=100):
  
  mask_pred=np.zeros((mask_size[1],mask_size[0]))
  count=0
  x_grid=center_size[0]//2
  y_grid=center_size[1]//2
  for j in range(0,mask_size[0]-window_size[0],stride):    
    for i in range(0,mask_size[1]-window_size[1],stride):  
      x_pos=i+window_size[0]//2
      y_pos=j+window_size[1]//2
      if count >= len(y_pred):
        count -= 1
      mask_pred[(x_pos-x_grid):(x_pos+x_grid),(y_pos-y_grid):(y_pos+y_grid)]=y_pred[count]
      count += 1
  return(mask_pred)

In [0]:
# Note: we should get all labels of slide rather than the center region here! 
# Otherwise, we would underestimate our evaluation.
def get_test_pred(model, slide, level, window_size=(299,299),
                     center_size=(128,128),stride=128):
  
  factor = int(slide.level_downsamples[level])
  preds = []
  for i in range(0, slide.level_dimensions[level][0]-window_size[0],stride):  
    for j in range(0, slide.level_dimensions[level][1]-window_size[1],stride): 
      slide_img = read_slide(slide, i*factor, j*factor, level, window_size[0], window_size[1], as_float=False)
      patches = kp_image.img_to_array(slide_img)/255
      y_pred = model.predict(np.expand_dims(patches.astype('float32'),axis=0))
      preds.append(y_pred)
  
  mask_pred = label2mask(np.asarray(preds), slide.level_dimensions[level], window_size = window_size, 
                         center_size=center_size, stride = stride)
  return mask_pred

In [0]:
mask_pred = get_test_pred(model_baseline, slide, level, window_size=window_size,
                          center_size=center_size,stride=stride)

In [0]:
slide_image = np.asarray(read_slide(slide, x=0, y=0, 
                         level=level, 
                         width=slide.level_dimensions[level][0], 
                         height=slide.level_dimensions[level][1]))
mask_image = np.asarray(read_slide(tumor_mask, x=0, y=0, 
                        level=level, 
                        width=slide.level_dimensions[level][0], 
                        height=slide.level_dimensions[level][1]))

In [0]:
#plt.figure(figsize=(10,10), dpi=50)
plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
plt.imshow(slide_image)
plt.imshow(mask_image[:,:,0],cmap='jet', alpha=0.5)
plt.grid(False)
plt.subplot(2,2,2)
plt.imshow(slide_image)
plt.imshow(mask_pred, cmap='jet',alpha=0.5) # Red regions contains cancer.
plt.grid(False)
plt.subplot(2,2,3)
plt.imshow(mask_image[:,:,0])
plt.imshow(mask_pred, cmap='jet',alpha=0.5) # Red regions contains cancer.
plt.grid(False)
plt.show()

## Step 4: Prediction for a New Slide

In [0]:
#Download a test slide
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

fname = 'tumor_078.tif'
if not os.path.exists(fname): 
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  f_ = drive.CreateFile({'id': '1cAFgPCkGR0zH0gqCDo4yBnstNoZmjp3C'})
  f_.GetContentFile(fname)
  fname = 'tumor_078_mask.tif'
  f_ = drive.CreateFile({'id': '1ZC2urznY3gRebUG3PN2BtYD1ZPZ7GPYq'})
  f_.GetContentFile(fname)

In [0]:
slide_test_path = 'tumor_078.tif'
test_mask_path = 'tumor_078_mask.tif'
slide_test = open_slide(slide_test_path)
mask_test = open_slide(test_mask_path)

In [0]:
mask_pred_test = get_test_pred(model_baseline, slide_test, level, window_size=window_size,
                               center_size=center_size,stride=stride)

In [0]:
slide_test_image = np.asarray(read_slide(slide_test, x=0, y=0, 
                              level=level, 
                              width=slide_test.level_dimensions[level][0], 
                              height=slide_test.level_dimensions[level][1]))
mask_test_image = np.asarray(read_slide(mask_test, x=0, y=0, 
                             level=level, 
                             width=slide_test.level_dimensions[level][0], 
                             height=slide_test.level_dimensions[level][1]))

In [0]:
#plt.figure(figsize=(10,10), dpi=50)
plt.figure(figsize=(12,10))
plt.subplot(2,2,1)
plt.imshow(slide_test_image)
plt.imshow(mask_test_image[:,:,0],cmap='jet', alpha=0.5)
plt.grid(False)
plt.subplot(2,2,2)
plt.imshow(slide_test_image)
plt.imshow(mask_pred_test, cmap='jet',alpha=0.5) # Red regions contains cancer.
plt.grid(False)
plt.subplot(2,2,3)
plt.imshow(mask_test_image[:,:,0])
plt.imshow(mask_pred_test, cmap='jet',alpha=0.5) # Red regions contains cancer.
plt.grid(False)
plt.show()

## Step 5: Evaluation

In [0]:
from sklearn import metrics
import sklearn

fpr, tpr, thresholds = metrics.roc_curve(mask_test_image[:,:,0].reshape(-1).astype('float32'), 
                                         mask_pred_test.reshape(-1), pos_label=None)

print('AUC:%s'%(metrics.auc(fpr, tpr)))

plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.plot(fpr, tpr)
plt.show()

In [0]:
thres = thresholds[np.argmax(1-fpr+tpr)]
mask_pred_test_labels = np.zeros(mask_pred_test.shape)
mask_pred_test_labels[mask_pred_test > thres] = 1.

In [0]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(mask_test_image[:,:,0].reshape(-1).astype('uint8'), 
                       mask_pred_test_labels.reshape(-1).astype('uint8'))

In [0]:
mat

In [0]:
recall = mat[1][1]/(mat[1][1]+mat[1][0])   #tp/tp+fn
precision = mat[1][1]/(mat[1][1]+mat[0][1])  #tp/tp+fp

In [0]:
print('Recall:{}'.format(recall))

In [0]:
print('Precision:{}'.format(precision))